In [ ]:
# Model Experimentation
# Only run once
import sys
import os

project_root = os.path.abspath('../')
os.chdir(project_root)

src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Setup

In [ ]:
from utils.load_data import DataLoader
from utils.preprocess import Preprocessor, FeatureEngineering
from utils.evaluate_model import Evaluator
from utils.model import ModelLoader
from selector.model_selector import ModelSelector
import pandas as pd
import numpy as np
import optuna
import mlflow
import os
import warnings
warnings.filterwarnings("ignore")

dl = DataLoader()
pre = Preprocessor()
fe = FeatureEngineering()
evaluator = Evaluator()
ml = ModelLoader()

In [ ]:
mlflow.set_tracking_uri("file:///c:/Documents/Fahmi/ai/ml/nama-folder-root/logs")

In [ ]:
train = dl.load_data('train_clean.csv', 'data/processed')
test = dl.load_data('test_clean.csv', 'data/processed')
submission = dl.load_data('sample_submission.csv', 'data/raw')

train_df = train.copy()
test_df = test.copy()

train_path = 'data/processed/train_clean.csv'
test_path = 'data/processed/test_clean.csv'

X_train, X_val, y_train, y_val = dl.split_data(train_df, id_column='id', target_column='target', test_size=0.2, random_state=42, stratify='target')

# Model selector

In [ ]:
selector = ModelSelector(data_path=train_path, target='target', is_classification=False, test_size=0.2, random_state=42)
selector.run()

# Feature engineering

In [ ]:
# Function

In [ ]:
# Execution

# Tuning hyperparameter

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Model

In [ ]:
# ml.save_model(model, ".pkl")

# Predict

In [ ]:
# model = ml.load_model(".pkl")
# pred = model.predict(test data)